In [1]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
from keras.callbacks import EarlyStopping
from keras.layers import Dropout
import re
from nltk.corpus import stopwords
from nltk import word_tokenize
STOPWORDS = set(stopwords.words('english'))
from bs4 import BeautifulSoup
import plotly.graph_objs as go
#import chart_studio.plotly as py
import cufflinks
from IPython.core.interactiveshell import InteractiveShell
import plotly.figure_factory as ff
InteractiveShell.ast_node_interactivity = 'all'
from plotly.offline import iplot
cufflinks.go_offline()
cufflinks.set_config_file(world_readable=True, theme='pearl')
import torch

/opt/conda/lib/python3.7/site-packages/geopandas/_compat.py:115: UserWarning:

The Shapely GEOS version (3.9.1-CAPI-1.14.2) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.



In [2]:
from keras.callbacks import ModelCheckpoint

In [3]:
# Train Data
df_train_en = pd.read_csv('../input/tamilmisodatanew/ta-en-misogyny-train.csv', sep='\t', header=None)
df_train_ta = pd.read_csv('../input/tamilmisodatanew/ta-misogyny-train.csv', sep='\t', header=None)

# Dev Data
df_dev_en = pd.read_csv('../input/tamilmisodatanew/ta-en-misogyny-dev.csv', sep='\t', header=None)
df_dev_ta = pd.read_csv('../input/tamilmisodatanew/ta-misogyny-dev.csv', sep='\t', header=None)

# Test Data
df_test_en = pd.read_csv('../input/tamilmisodatanew/ta-en-misogyny-test.csv', sep='\t', header=None)
df_test_ta = pd.read_csv('../input/tamilmisodatanew/ta-misogyny-test.csv', sep='\t', header=None)

In [4]:
df_train_ea = pd.read_csv('../input/emotion-analysis-in-tamil/Emotion_Analysis_Task_A_train.csv', sep = '\t', header = None)
df_dev_ea = pd.read_csv('../input/emotion-analysis-in-tamil/Emotion_Analysis_Task_A_dev.csv', sep = '\t', header = None)
df_test_ea = pd.read_csv('../input/emotion-analysis-in-tamil/Emotion_Analysis_Task_A_test.csv', sep = '\t', header = None)


In [5]:
train_data = df_train_ta.dropna()
val_data = df_dev_ta.dropna()
test_data = df_test_ta.dropna()


In [6]:
train_data.drop([1184,1922], inplace = True)

In [7]:
#train_data[0].value_counts()

In [8]:
# The maximum number of words to be used. (most frequent)
MAX_NB_WORDS = 64000
# Max number of words in each complaint.
MAX_SEQUENCE_LENGTH = 128
# This is fixed.
EMBEDDING_DIM = 100
tokenizer = Tokenizer(num_words=MAX_NB_WORDS, filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~', lower=False)
tokenizer.fit_on_texts(train_data[1])
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

Found 11235 unique tokens.


In [9]:
X_train = tokenizer.texts_to_sequences(train_data[1])
X_train = pad_sequences(X_train, maxlen=MAX_SEQUENCE_LENGTH)
print('Shape of data tensor:', X_train.shape)

X_val = tokenizer.texts_to_sequences(val_data[1])
X_val = pad_sequences(X_val, maxlen=MAX_SEQUENCE_LENGTH)
print('Shape of data tensor:', X_val.shape)

X_test = tokenizer.texts_to_sequences(test_data[1])
X_test = pad_sequences(X_test, maxlen=MAX_SEQUENCE_LENGTH)
print('Shape of data tensor:', X_test.shape)

Shape of data tensor: (2238, 128)
Shape of data tensor: (560, 128)
Shape of data tensor: (699, 128)


In [10]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 9376656373477428653
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 15667560448
locality {
  bus_id: 1
  links {
  }
}
incarnation: 12831235706792062458
physical_device_desc: "device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0"
]


2022-03-08 04:44:37.383765: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-03-08 04:44:37.437477: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-03-08 04:44:37.438571: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-03-08 04:44:37.439294: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA 

In [11]:
Y_train = pd.get_dummies(train_data[0]).values
print('Shape of label tensor:', Y_train.shape)

Y_val = pd.get_dummies(val_data[0]).values
print('Shape of label tensor:', Y_val.shape)

Shape of label tensor: (2238, 8)
Shape of label tensor: (560, 8)


In [12]:
def get_predicted(preds):
    pred_flat = np.argmax(preds, axis=1).flatten()
    return pred_flat
from sklearn.metrics import f1_score, accuracy_score
# def f1(ytrue, ypred):
#     return f1_score(get_predicted(np.array(ytrue)), get_predicted(np.array(ypred)), average = 'weighted')

In [13]:
from keras import backend as K
def f1(true, pred): #shapes (batch, 4)

    #for metrics include these two lines, for loss, don't include them
    #these are meant to round 'pred' to exactly zeros and ones
    #predLabels = K.argmax(pred, axis=-1)
    #pred = K.one_hot(predLabels, 4) 


    ground_positives = K.sum(true, axis=0)       # = TP + FN
    pred_positives = K.sum(pred, axis=0)         # = TP + FP
    true_positives = K.sum(true * pred, axis=0)  # = TP
        #all with shape (4,)

    precision = (true_positives + K.epsilon()) / (pred_positives + K.epsilon()) 
    recall = (true_positives + K.epsilon()) / (ground_positives + K.epsilon()) 
        #both = 1 if ground_positives == 0 or pred_positives == 0
        #shape (4,)

    f1 = 2 * (precision * recall) / (precision + recall + K.epsilon())
        #not sure if this last epsilon is necessary
        #matematically not, but maybe to avoid computational instability
        #still with shape (4,)

    weighted_f1 = f1 * ground_positives / K.sum(ground_positives)
    weighted_f1 = K.sum(weighted_f1)


    return weighted_f1 #for metrics, return only 'weighted_f1

In [14]:
mc = ModelCheckpoint('model_tam_1.h5', monitor='val_f1', mode='max', save_best_only=True)
cb_list = mc

In [15]:
from tensorflow.keras import layers

In [16]:
model = Sequential()
model.add(Embedding(MAX_NB_WORDS, EMBEDDING_DIM, input_length=X_train.shape[1]))
model.add(SpatialDropout1D(0.2))
model.add(layers.Bidirectional(LSTM(100, dropout=0.2, recurrent_dropout=0.2)))
model.add(Dense(8, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=[f1])
history = model.fit(X_train, Y_train, validation_data=(X_val, Y_val), epochs=4, batch_size=32, callbacks = cb_list)
# Final evaluation of the model
model_pred_train = model.predict(X_train)
model_pred_test = model.predict(X_val)
# print(classification_report(test_labels,model_pred_test))
#print('LSTM Recurrent Neural Network baseline: ' + str(roc_auc_score(Y_train, model_pred_train)))
#print('LSTM Recurrent Neural Network: ' + str(roc_auc_score(Y_val, model_pred_test)))

2022-03-08 04:44:42.710406: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-03-08 04:44:42.711301: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-03-08 04:44:42.711941: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-03-08 04:44:42.712845: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-03-08 04:44:42.713464: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from S

Epoch 1/4
70/70 [==============================] - 87s 1s/step - loss: 1.3871 - f1: 0.3773 - val_loss: 1.1920 - val_f1: 0.4313
Epoch 2/4
70/70 [==============================] - 80s 1s/step - loss: 1.1033 - f1: 0.4806 - val_loss: 1.0323 - val_f1: 0.5578
Epoch 3/4
70/70 [==============================] - 79s 1s/step - loss: 0.6955 - f1: 0.6698 - val_loss: 1.1293 - val_f1: 0.5844
Epoch 4/4
70/70 [==============================] - 78s 1s/step - loss: 0.4361 - f1: 0.7788 - val_loss: 1.3818 - val_f1: 0.5893


In [17]:

model_pred_train = model.predict(X_train)
model_pred_test = model.predict(X_val)
f1 = f1_score(get_predicted(Y_train), get_predicted(model_pred_train), average = 'weighted')
acc = accuracy_score(get_predicted(Y_train), get_predicted(model_pred_train))
print(f1, acc)

0.8872764486306678 0.90437890974084


In [18]:
f1 = f1_score(get_predicted(Y_val), get_predicted(model_pred_test), average = 'macro')
f1

0.20582202322741483

In [19]:
acc = accuracy_score(get_predicted(Y_val), get_predicted(model_pred_test))
print(f1, acc)

0.20582202322741483 0.6589285714285714


In [20]:
for x in get_predicted(Y_train): print(x)

2
5
5
5
1
5
5
5
3
1
5
5
5
5
3
5
5
5
0
5
5
3
5
4
5
5
5
5
5
5
5
5
0
7
0
6
4
5
2
5
5
0
2
5
5
5
5
4
5
5
5
3
3
5
5
3
5
0
4
5
3
5
3
5
3
3
5
2
5
3
5
7
5
5
5
5
5
3
5
0
3
5
3
5
5
3
3
3
5
3
4
3
3
5
5
3
5
5
5
0
0
3
5
7
3
3
5
5
5
5
5
3
2
5
5
3
0
5
3
3
5
5
5
7
5
5
5
5
5
0
5
5
3
5
0
5
5
5
5
5
5
3
2
5
2
4
5
5
5
5
5
3
5
5
0
4
3
5
3
5
5
4
2
7
5
5
5
5
5
7
7
1
4
4
5
5
5
2
7
1
3
5
3
5
5
4
5
5
5
3
5
5
5
5
5
5
5
5
5
5
5
3
3
5
5
5
5
5
5
3
3
5
5
3
5
0
5
5
3
5
5
5
5
3
4
5
5
5
5
5
5
5
5
7
0
3
5
5
0
5
5
5
3
5
7
2
5
5
5
3
2
0
5
5
5
3
5
3
1
5
3
5
5
3
2
3
5
5
3
0
4
3
5
5
5
5
5
5
4
3
5
3
5
5
5
3
5
2
5
5
5
2
4
5
7
3
5
3
4
3
5
0
5
3
0
5
5
5
5
3
5
0
1
5
5
5
5
3
5
5
3
5
5
5
5
3
5
4
7
3
5
5
3
3
3
5
0
4
5
5
3
2
2
5
5
3
7
7
0
5
5
5
5
4
5
0
5
5
5
5
6
3
3
5
5
5
3
5
3
5
5
5
3
0
5
5
5
0
5
5
3
3
0
2
4
5
5
5
3
5
5
5
5
5
5
5
3
3
5
3
1
5
3
5
2
5
3
3
3
3
5
5
5
5
5
7
5
5
5
5
3
0
0
3
5
5
3
5
7
5
2
5
5
5
5
5
3
5
2
0
5
5
5
3
5
4
5
5
5
5
5
5
3
4
3
5
5
5
5
5
5
5
3
5
5
4
5
0
5
5
3
5
3
5
5
5
5
5
5
5
5
5
4
3
2
3
5
5
1
5
5
0
5
5
3
5
3
5
3
5


In [21]:
from sklearn.metrics import classification_report
report = classification_report(get_predicted(Y_val), get_predicted(model_pred_test),output_dict=True)


/opt/conda/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

/opt/conda/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

/opt/conda/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.



In [22]:
pd.DataFrame(report).transpose()

,precision,recall,f1-score,support
0,0.363636,0.111111,0.170213,36.000000
1,0.000000,0.000000,0.000000,8.000000
2,0.000000,0.000000,0.000000,11.000000
3,0.487179,0.548077,0.515837,104.000000
4,0.214286,0.125000,0.157895,24.000000
5,0.736715,0.881503,0.802632,346.000000
6,0.000000,0.000000,0.000000,2.000000
7,0.000000,0.000000,0.000000,29.000000
accuracy,0.658929,0.658929,0.658929,0.658929
macro avg,0.225227,0.208211,0.205822,560.000000
